In [18]:
# Module Importations
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Print versioning information
print(sklearn.__version__, np.__version__, pd.__version__) 

0.22.2.post1 1.19.4 1.1.4


In [19]:
# Custom Module Imports
from Source.data import load_data
from Source.data import split_data

In [20]:
# Constants

In [21]:
# Load data from pickle
original_dataset_df = load_data.load_pickled_data('full_data_df.pkl')

Loaded pickled dataframe ...


In [22]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_eval(original_dataset_df, 0.2)

Original Data Items: 137556
Training Data Items: 110045
Evaluation Data Items: 27511


In [23]:
# Drop unrequired data columns

# Identify columns to drop 
columns_to_drop = ['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']

training_set = training_set.drop(columns_to_drop, axis = 1)

print(training_set.head(5))

        DC_POWER CELL_NO TIME_OF_DAY   AMB_TEMP   MOD_TEMP  IRRADIATION  \
99361      0.000      17    19:00:00  32.823653  31.564240     0.000000   
113108  3576.750       1    09:00:00  28.047660  43.362591     0.686030   
53656      0.000      14    19:30:00  24.490542  22.900633     0.000000   
46387   9456.625       6    09:00:00  23.982949  38.816762     0.630916   
116927     0.000      22    05:00:00  24.222456  23.379348     0.000000   

         PLANT  
99361   plant2  
113108  plant2  
53656   plant1  
46387   plant1  
116927  plant2  


In [24]:
# Create DC Power Target datasets

# 